In [1]:
import os
import sys
import pandas as pd
import pandasql as pdsql


from bkcharts import Donut
from bokeh.io import output_notebook, show

# Do this trick to add the new Beamly functionality
pythonModsRoot = '/Users/carlos.aguilar/Google Drive/PythonDev/Coding/BeamlyPython'
if pythonModsRoot not in sys.path:
    sys.path.append(pythonModsRoot)

import redshiftSqlAlchemy as rsa
import carlosUtils        as cu
import bokehUtils         as bk

from IPython.display import Markdown, display, HTML
def printmd(string):
    display(Markdown(string))


In [2]:
numCampaigns = 20;
cutOffPerc   = 96.0;

# set the table and schema
schemaName = 'adform'
tableName  = 'clickExtended'

# set the name of the variables to break-down
breakDownKeyWords = ['campaign_id', 'yyyy_mm_dd', 'click_detail_id_paid_keyword_id', \
'publisher_domain', 'destination_url', 'bannerid', 'bannerType', \
'client_id', 'device_type_id', 'placement_id_activity_id', 'tag_id', \
'city_id' ];

# flags to set the outputs
saveFigure    = True;
saveDFToExcel = True;

In [11]:
# connect to the Redshift box
user     = 'carlos_aguilar'
password = 'MdogDI64j6vH90g973'
dbname   = 'adform'
host     = 'adform-ops.c7dxcjhlundm.eu-central-1.redshift.amazonaws.com'

rs = rsa.RedshiftAlchemy(user=user, password=password, 
    database=dbname, host=host)

In [12]:
sqlQuery = '''SELECT count(*) as numRecords 
    from {}.{}'''.format(schemaName, tableName)
df = rs.query2DF(sqlQuery);
printmd('Analysing **' + schemaName + '.' + tableName + '** (' +  str(df.numrecords[0]) + ' records)' )

Analysing **adform.clickExtended** (1947522 records)

In [13]:
# This snippet breaks down the variables and writes the counts to a file
printmd('_Break down variables_')
for currentKey in breakDownKeyWords:
    sqlQuery = '''SELECT count(distinct({})) as numRecords 
        from {}.{}'''.format(currentKey, schemaName, tableName)
    df = rs.query2DF(sqlQuery);
    currentText = '**' + currentKey + '**' + ' has got **' + str(df.numrecords[0]) + '** unique values...'
    printmd(currentText)

_Break down variables_

**campaign_id** has got **235** unique values...

**yyyy_mm_dd** has got **66** unique values...

**click_detail_id_paid_keyword_id** has got **1519** unique values...

**publisher_domain** has got **8679** unique values...

...

**client_id** has got **39** unique values...

**device_type_id** has got **5** unique values...

**placement_id_activity_id** has got **1521** unique values...

**tag_id** has got **2549** unique values...

**city_id** has got **82186** unique values...

In [14]:
# This bit takes the 'numCampaigns' most clicked campaigns and writes the results
sqlQuery = '''SELECT
A.campaign_id,
A.clientsname,
A.countryname,
A.devicename,
A.startdate,
A.enddate,
sum(A.numRecords) as totalClicks,
B.numTotals
from adform.clickExtended as A
inner join (
  SELECT  campaign_id,
  sum(numRecords) AS numTotals,
  rank() over (order by numTotals desc) as rnk
  from adform.clickExtended
  group by 1
) as B
on A.campaign_id = B.campaign_id
where B.rnk <= {}
group by 1,2,3,4,5,6,B.numTotals
order by totalclicks DESC'''.format(numCampaigns)


df        = rs.query2DF(sqlQuery)
groupedDF = df.groupby(df['campaign_id'])
idx       = 0;


# Get a list of the most clicked campaigns
vars2show = ['campaign_id', 'clientsname', 'numtotals', 'startdate', 'enddate']
df2 = df[vars2show].copy()
df2.drop_duplicates(inplace=True)
display(df2)

,campaign_id,clientsname,numtotals,startdate,enddate
0,919044,Gucci,282480,2017-06-04 00:00:00.0,2017-06-30 23:59:00.0
1,897248,CoverGirl,171958,2017-05-15 00:00:00.0,2017-06-30 23:59:00.0
2,886811,Sally Hansen,171157,2017-04-25 00:00:00.0,2017-06-30 23:59:00.0
3,929026,Sally Hansen,114949,2017-06-12 00:00:00.0,2017-06-30 23:59:00.0
4,918140,Wella Professional,106090,2017-06-02 00:00:00.0,2017-06-30 23:59:00.0
5,950281,Sally Hansen,83873,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0
6,906824,Wella Retail,70696,2017-05-25 00:00:00.0,2017-06-30 23:59:00.0
7,922742,Hugo Boss,83245,2017-06-05 00:00:00.0,2017-07-23 23:59:00.0
8,912694,OPI Professional,67760,2017-06-01 00:00:00.0,2017-06-30 23:59:00.0
10,894699,Rimmel,60649,2017-05-10 00:00:00.0,2017-06-06 23:59:00.0


In [15]:
for name, group in groupedDF:
    idx += 1;
    groupClientName = group['clientsname'].iloc[0]
    cu.printf('{} - Processing {} id {}...'.format(idx, groupClientName, name))
    print('Current campaign {} id {} got {} clicks'.format(groupClientName, \
        name, group['totalclicks'].sum()))
    
    # Get the percentage for the break down
    totalCampaignClicks = group['totalclicks'].sum();
    group['perct'] = 100.0*group['totalclicks']/totalCampaignClicks;
    
    # cutoff at 96%
    idx95p = group['perct'].cumsum() < cutOffPerc
    df95p  = group.loc[idx95p, :]

    fileName  = groupClientName + ' (id ' + str(name) + ').xlsx'
    
    display(df95p)

    # bokeh
    title  = groupClientName + \
        ' (id ' + str(name) + ') clicks: ' + str(totalCampaignClicks)
    labels = ['devicename','countryname']
    values = 'totalclicks'
    hoverText ='totalclicks'
    textFontSize = '10pt'

    donutChart = Donut(df95p, label=labels, values=values,
    text_font_size=textFontSize, hover_text=hoverText,
    title=title)
    output_notebook()
    show(donutChart)

Current campaign Philosophy id 875443 got 65436 clicks


1 - Processing Philosophy id 875443...
/Users/carlos.aguilar/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,campaign_id,clientsname,countryname,devicename,startdate,enddate,totalclicks,numtotals,perct
11,875443,Philosophy,United States,Mobile,2017-04-06 00:00:00.0,2017-06-30 23:59:00.0,52534,65436,80.283025
35,875443,Philosophy,United States,Tablet,2017-04-06 00:00:00.0,2017-06-30 23:59:00.0,6449,65436,9.855431


/Users/carlos.aguilar/anaconda3/lib/python3.6/site-packages/bkcharts/builders/donut_builder.py:118: PerformanceWarning: indexing past lexsort depth may impact performance.
  chart = create_and_build(DonutBuilder, data, **kw)


Loading BokehJS ...

...

Loading BokehJS ...

Current campaign Sally Hansen id 950281 got 83873 clicks


20 - Processing Sally Hansen id 950281...


,campaign_id,clientsname,countryname,devicename,startdate,enddate,totalclicks,numtotals,perct
5,950281,Sally Hansen,United States,Mobile,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,72795,83873,86.791935
36,950281,Sally Hansen,United States,Tablet,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,6203,83873,7.395705


Loading BokehJS ...

In [16]:


# Assuming that dataframes df1 and df2 are already defined:

display(df)
print('Dataframe 2:')
HTML(df.to_html())

,campaign_id,clientsname,countryname,devicename,startdate,enddate,totalclicks,numtotals
0,919044,Gucci,Italy,Mobile,2017-06-04 00:00:00.0,2017-06-30 23:59:00.0,180819,282480
1,897248,CoverGirl,United States,Mobile,2017-05-15 00:00:00.0,2017-06-30 23:59:00.0,140134,171958
2,886811,Sally Hansen,United States,Mobile,2017-04-25 00:00:00.0,2017-06-30 23:59:00.0,108137,171157
3,929026,Sally Hansen,United States,Mobile,2017-06-12 00:00:00.0,2017-06-30 23:59:00.0,101296,114949
4,918140,Wella Professional,United Kingdom,Mobile,2017-06-02 00:00:00.0,2017-06-30 23:59:00.0,89825,106090
5,950281,Sally Hansen,United States,Mobile,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,72795,83873
6,906824,Wella Retail,United Kingdom,Mobile,2017-05-25 00:00:00.0,2017-06-30 23:59:00.0,66829,70696
7,922742,Hugo Boss,Poland,Desktop and Laptop,2017-06-05 00:00:00.0,2017-07-23 23:59:00.0,66030,83245
8,912694,OPI Professional,United States,Mobile,2017-06-01 00:00:00.0,2017-06-30 23:59:00.0,59063,67760
9,886811,Sally Hansen,United States,Desktop and Laptop,2017-04-25 00:00:00.0,2017-06-30 23:59:00.0,56824,171157


Dataframe 2:


,campaign_id,clientsname,countryname,devicename,startdate,enddate,totalclicks,numtotals
0,919044,Gucci,Italy,Mobile,2017-06-04 00:00:00.0,2017-06-30 23:59:00.0,180819,282480
1,897248,CoverGirl,United States,Mobile,2017-05-15 00:00:00.0,2017-06-30 23:59:00.0,140134,171958
2,886811,Sally Hansen,United States,Mobile,2017-04-25 00:00:00.0,2017-06-30 23:59:00.0,108137,171157
3,929026,Sally Hansen,United States,Mobile,2017-06-12 00:00:00.0,2017-06-30 23:59:00.0,101296,114949
4,918140,Wella Professional,United Kingdom,Mobile,2017-06-02 00:00:00.0,2017-06-30 23:59:00.0,89825,106090
5,950281,Sally Hansen,United States,Mobile,2017-07-05 00:00:00.0,2017-07-31 23:59:00.0,72795,83873
6,906824,Wella Retail,United Kingdom,Mobile,2017-05-25 00:00:00.0,2017-06-30 23:59:00.0,66829,70696
7,922742,Hugo Boss,Poland,Desktop and Laptop,2017-06-05 00:00:00.0,2017-07-23 23:59:00.0,66030,83245
8,912694,OPI Professional,United States,Mobile,2017-06-01 00:00:00.0,2017-06-30 23:59:00.0,59063,67760
9,886811,Sally Hansen,United States,Desktop and Laptop,2017-04-25 00:00:00.0,2017-06-30 23:59:00.0,56824,171157
